# ⚙️ 환경 설정 및 라이브러리 설치

프로젝트 실행에 필요한 파이썬 라이브러리들을 설치합니다. 각 라이브러리의 역할은 다음과 같습니다.

  * **`langchain`, `langgraph`, `langchain-community`**: LangChain 및 LangGraph의 핵심 기능을 사용하기 위한 라이브러리입니다.
  * **`langchain-google-genai`**: Google Gemini와 같은 생성형 AI 모델을 LangChain과 연동합니다.
  * **`tavily-python`**: 실시간 정보 검색을 위한 Tavily 검색 도구를 추가합니다.
  * **`langchain-mcp-adapters`**: 특정 어댑터(MCP)를 사용하기 위해 필요합니다.

아래 셀을 실행하여 모든 라이브러리를 한 번에 설치하세요.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [19]:
!pip install langchain==0.3.27 langgraph==0.6.6 langchain-google-genai==2.1.9 langchain-community==0.3.27 langchain-mcp-adapters==0.1.9 tavily-python==0.7.11

👇 아래 버튼을 눌러 코드를 실행합니다.

In [20]:
import os
import time
import textwrap
from pprint import pprint
from typing import Dict, Any

from IPython.display import display, HTML
from langchain_core.messages import HumanMessage, BaseMessage

# ----------------------------------------------------
# ### ⚙️ 1. MCP 서버 관리 공통 함수 ###
# ----------------------------------------------------

def start_mcp_server(script_name: str, port: int, log_file: str, service_name: str) -> bool:
    """
    공통 MCP 서버 시작 및 확인 함수

    Args:
        script_name: 실행할 Python 스크립트 파일명 (예: "search_server.py")
        port: 사용할 포트 번호 (예: 8000)
        log_file: 저장할 로그 파일명 (예: "search_server.log")
        service_name: 로그 출력 시 사용할 서비스 이름 (예: "금융 정보")
    """
    print(f"🚀 {service_name} MCP 서버를 시작합니다... (Port: {port})")

    # 포트를 사용하는 기존 프로세스 종료 및 새 서버 시작
    server_command = (
        f"lsof -ti tcp:{port} | xargs -r kill; "  # 기존 프로세스 종료
        f"rm -f {log_file}; "                       # 이전 로그 파일 삭제
        f"nohup python -u {script_name} > {log_file} 2>&1 &"  # 백그라운드 실행
    )
    os.system(server_command)

    # --- 서버 로그 파일 확인 및 상태 출력 ---
    timeout_seconds = 30  # 최대 대기 시간 (초)
    check_interval = 2   # 로그 확인 주기 (초)
    start_time = time.time()
    server_status = "PENDING"
    log_content = ""

    print(f"⏱️  서버 시작 대기 중... (최대 {timeout_seconds}초)")

    while time.time() - start_time < timeout_seconds:
        try:
            with open(log_file, 'r') as f:
                log_content = f.read()

            log_content_lower = log_content.lower()

            # 성공 조건 확인
            if "uvicorn running on" in log_content_lower and "application startup complete" in log_content_lower:
                server_status = "SUCCESS"
                break
            # 실패 조건 확인
            if "traceback" in log_content_lower or "error" in log_content_lower:
                server_status = "FAILED"
                break

            time.sleep(check_interval)

        except FileNotFoundError:
            time.sleep(check_interval)
        except Exception as e:
            print(f"로그 파일 읽기 중 예외 발생: {e}")
            log_content = f"로그 파일 읽기 오류: {e}"
            server_status = "FAILED"
            break

    # --- 최종 결과 출력 ---
    print("-" * 30)
    if server_status == "SUCCESS":
        print(f"✅ {service_name} MCP 서버가 성공적으로 시작되어 실행 중입니다.")
        print(f"   (로그 파일: {os.getcwd()}/{log_file})")
        return server_status
    elif server_status == "FAILED":
        print(f"❌ {service_name} 서버 실행 중 오류가 발생했습니다. 아래 로그를 확인하세요.")
        print("-" * 20)
        print(log_content)
        print("-" * 20)
    else: # PENDING (Timeout)
        print(f"🤔 {timeout_seconds}초 내에 {service_name} 서버 시작 메시지를 확인하지 못했습니다.")
        print("\n   [현재 로그 파일 내용]")
        print("-" * 20)
        print(log_content if log_content else "[로그 파일 비어 있거나 생성되지 않았습니다]")
        print("-" * 20)

    return server_status

def stop_mcp_server(port: int, service_name: str):
    """지정된 포트의 MCP 서버를 종료합니다."""
    print(f"\n🛑 {service_name} MCP 서버(Port: {port})를 종료합니다...")
    os.system(f"lsof -ti tcp:{port} | xargs -r kill")
    print(f"✅ {service_name} 서버가 종료되었습니다.")

# ----------------------------------------------------
# ### 📊 2. Mermaid 그래프 시각화 공통 함수 ###
# ----------------------------------------------------

def display_mermaid_graph(app: Any, title: str = "Agent 워크플로우"):
    """
    컴파일된 LangGraph app의 Mermaid 다이어그램을 Colab에 시각화합니다.

    Args:
        app: LangGraph.compile()로 생성된 실행 가능한 객체
        title: 다이어그램 위에 표시할 제목
    """
    print(f"\n📊 [ {title} ]")

    # 1. app 객체에서 Mermaid 다이어그램 문법(텍스트)을 생성합니다.
    mermaid_syntax = app.get_graph().draw_mermaid()

    # 2. 이 Mermaid 텍스트를 HTML과 Javascript로 감싸서 렌더링할 준비를 합니다.
    html_code = f"""
    <div style="background-color: white; border: 1px solid #ddd; padding: 10px; border-radius: 5px;">
        <pre class="mermaid">{textwrap.dedent(mermaid_syntax)}</pre>
    </div>
    <script type="module">
        import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
        await mermaid.run();
    </script>
    """

    # 3. Colab/Jupyter 환경에서 HTML 코드를 실행하여 다이어그램을 출력합니다.
    display(HTML(html_code))
    print()

# API 키 설정 가이드

API를 사용하기 위한 키 설정 방법을 안내합니다. 각 서비스별로 아래 절차를 따라주세요.

<br>

## ✨ Gemini API 키 설정

1.  **API 키 발급받기**
    Gemini API를 사용하려면 먼저 API 키가 필요합니다. Google AI Studio에서 클릭 한 번으로 간편하게 키를 생성하세요.

    <a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">API 키 받기</a>

2.  **Colab에 키 저장하기**
    Colab의 왼쪽 패널에서 "🔑" 아이콘(보안 비밀)을 열어, 발급받은 키를 **`GOOGLE_API_KEY`** 라는 이름으로 저장하세요.

3.  **코드에서 키 사용하기**
    SDK는 **환경 변수**로 저장된 키를 자동으로 인식하여 사용합니다. Colab 보안 비밀에 저장한 키는 아래와 같이 불러올 수 있습니다.

    ```python
    # SDK가 자동으로 키를 인식합니다
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    ```

<br>

## 🔍 Tavily API 키 설정

1.  **API 키 발급받기**
    Tavily 검색 API를 사용하려면 API 키를 발급받아야 합니다. Tavily 대시보드에서 키를 생성하고 복사하세요.

    <a class="button button-primary" href="https://app.tavily.com" target="_blank" rel="noopener noreferrer">Tavily API 키 받기</a>

    > ✅ **참고:** Tavily는 무료 플랜을 제공하여 간단한 테스트나 소규모 프로젝트를 부담 없이 시작할 수 있습니다.

2.  **Colab에 키 저장하기**
    Colab의 "🔑" 아이콘(보안 비밀)을 열어, 발급받은 키를 **`TAVILY_API_KEY`** 라는 이름으로 저장하세요.

3.  **코드에서 키 사용하기**
    Tavily SDK 역시 **환경 변수**로 저장된 키를 자동으로 찾아 사용합니다.

    ```python
    # SDK가 자동으로 키를 인식합니다
    os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
    ```

👇 아래 버튼을 눌러 코드를 실행합니다.

In [21]:
import os
from google.colab import userdata

# Gemini API 키 설정
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Tavily API 키 설정
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

print("API 키 설정이 완료되었습니다.")

API 키 설정이 완료되었습니다.


# 🔍 검색 도구(Tavily) 정의 및 테스트

이 단계에서는 **Tavily 검색 API**를 사용하는 LangChain 검색 도구를 정의하고, 정상적으로 작동하는지 테스트합니다.<br>
Tavily 도구는 Agent가 외부 최신 정보에 접근해야 할 때 사용됩니다.

  * **도구 정의**: `TavilySearchResults`를 가져와 `search_tool`이라는 이름의 도구를 생성합니다.
  * **결과 수 제한**: `max_results=3` 파라미터를 설정하여, 검색 시 최대 3개의 관련성 높은 결과만 반환하도록 제한합니다.
  * **테스트 실행**: `"삼성전자 현재 주가"`와 같은 실제 검색어를 입력하여 도구가 외부 정보를 잘 가져오는지 확인합니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults
from pprint import pprint

# Tavily 검색 도구를 정의하고, 반환 결과의 수를 3개로 제한합니다.
search_tool = TavilySearchResults(max_results=3)

# 정의된 도구가 잘 작동하는지 'invoke'를 통해 테스트합니다.
results = search_tool.invoke({"query": "삼성전자 현재 주가"})
pprint(results)

[{'content': 'PLTR\n'
             '\n'
             '+3.04%\n'
             '\n'
             '## 자주 묻는 질문\n'
             '\n'
             '### 오늘 삼성전자 (005930)의 주가는?\n'
             '\n'
             '오늘 삼성전자 주가는 107,500입니다.\n'
             '\n'
             '### 삼성전자은(는) 어떤 증권 거래소에서 거래되나요?\n'
             '\n'
             '삼성전자은(는) 서울 증권 거래소 증권 거래소에 상장되어 거래되고 있습니다.\n'
             '\n'
             '### 삼성전자의 주식 티커는?\n'
             '\n'
             '삼성전자의 주식 티커는 “005930”입니다.\n'
             '\n'
             '### 삼성전자은(는) 배당을 지급하나요? 현재 배당수익률은 얼마인가요?\n'
             '\n'
             '삼성전자 배당수익률은 1.38%입니다.\n'
             '\n'
             '### 삼성전자 시가총액은 얼마인가요?\n'
             '\n'
             '오늘 현재 삼성전자 시가총액은 700.06T입니다.\n'
             '\n'
             '### 삼성전자의 주당순이익(TTM)은 얼마인가요?\n'
             '\n'
             '삼성전자의 EPS(TTM)는 4,606.36입니다.\n'
             '\n'
             '### 삼성전자의 다음 실적 발표일은 언제인가요?\n'
             '\n'
             '삼성전자은(는) 2026년 1월 28일에 다음 실적 보고서

# 🤖 Agent 설정 및 상태(State) 정의

Agent의 핵심 구성 요소를 설정합니다.<br>
Agent의 **'두뇌'** 역할을 하는 LLM(거대 언어 모델)과, 작업 흐름 전반에 걸쳐 정보를 저장하고 공유하는 **'메모리'** 역할의 상태(State)를 정의합니다.

  * **LLM (두뇌) 설정** : Agent의 추론과 판단을 담당할 언어 모델로 Google의 `gemini-2.0-flash-lite`를 설정합니다. 이 모델은 빠른 응답 속도와 우수한 성능을 균형 있게 제공합니다.
  * **AgentState (메모리) 정의** : Agent가 각 단계를 거치며 얻는 정보를 저장하고 공유할 데이터 구조인 `AgentState`를 정의합니다. LangGraph에서는 이 **상태(State)** 객체를 통해 각 노드(작업 단위)가 정보를 주고받습니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# 1. Agent의 '두뇌' 역할을 할 LLM을 설정합니다.
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

# 2. Agent의 작업 내용을 기억하고 공유할 '메모리' 구조(State)를 정의합니다.
class AgentState(TypedDict):
    """
    Agent의 상태를 정의합니다. 모든 노드는 이 상태를 공유하며 정보를 업데이트합니다.

    Attributes:
        company_name: 분석할 회사의 이름 (사용자 입력)
        stock_price: 검색된 주가 정보
        news: 검색된 관련 뉴스 목록
    """
    company_name: str
    stock_price: str
    news: List[str]

# 🧩 Agent 작업 노드(Node) 함수 정의

Agent 워크플로우를 구성하는 각 노드(Node)의 실제 동작을 함수로 정의합니다.<br>
각 함수는 **`AgentState`** 를 입력받아 특정 작업을 수행하고, 그 결과를 다시 **`AgentState`** 에 저장하거나 최종 결과를 생성하는 역할을 합니다.

  * **`stock_price_search_node`** : `AgentState`에서 **회사 이름**을 가져와 **주가**를 검색하고, 결과를 상태에 업데이트합니다.
  * **`news_search_node`** : `AgentState`에서 **회사 이름**을 가져와 **최신 뉴스 3개**를 검색하고, 결과를 상태에 업데이트합니다.
  * **`analysis_node`** : 모든 정보가 채워진 `AgentState`를 기반으로, **LLM에게 전달할 상세 프롬프트**를 생성합니다. LLM은 이 프롬프트를 바탕으로 최종 분석 리포트를 생성하며, 이 노드는 상태를 추가로 업데이트하지 않고 리포트를 출력하는 것으로 작업을 마칩니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [24]:
def stock_price_search_node(state: AgentState):
    """주어진 회사 이름으로 현재 주가를 검색하는 노드"""
    print("--- (1/3) 주가 정보 검색 중... ---")
    company_name = state["company_name"]

    # Tavily 도구를 사용하여 주가 정보 검색
    stock_price = search_tool.invoke({"query": f"{company_name} 현재 주가"})
    return {"stock_price": stock_price}

def news_search_node(state: AgentState):
    """주어진 회사 이름으로 최신 뉴스를 검색하는 노드"""
    print("--- (2/3) 관련 뉴스 검색 중... ---")
    company_name = state["company_name"]

    # Tavily 도구를 사용하여 최신 뉴스 3개 검색
    news = search_tool.invoke({"query": f"{company_name} 최신 주요 뉴스 3개"})
    return {"news": news}

def analysis_node(state: AgentState):
    """수집된 정보를 바탕으로 종합 분석 리포트를 생성하는 노드"""
    print("--- (3/3) 종합 분석 리포트 생성 중... ---")
    company_name = state["company_name"]
    stock_price = state["stock_price"]
    news = state["news"]

    # Gemini LLM에게 전달할 프롬프트(지시문) 작성
    prompt = f"""
    당신은 전문 주식 애널리스트입니다. 다음 정보를 바탕으로 '{company_name}'에 대한 투자 분석 리포트를 작성해주세요.

    **[분석 정보]**
    1. **현재 주가**: {stock_price}
    2. **관련 최신 뉴스**: {news}

    **[리포트 작성 가이드라인]**
    - **개요**: 현재 상황을 간결하게 요약합니다.
    - **긍정적 요인**: 주가에 긍정적인 영향을 줄 수 있는 요소를 뉴스 내용에 기반하여 분석합니다.
    - **부정적 요인**: 주가에 부정적인 영향을 줄 수 있는 요소를 뉴스 내용에 기반하여 분석합니다.
    - **종합 의견 및 투자 전략**: 위 내용을 종합하여 최종 투자 의견과 간단한 전략을 제시해주세요. 전문가적이지만 이해하기 쉬운 톤으로 작성해주세요.

    **주의**: 이 리포트는 정보 제공 및 교육적 목적으로 작성되었으며, 실제 투자 권유가 아님을 반드시 명시해주세요.
    """

    # LLM을 호출하여 분석 리포트 생성
    response = llm.invoke(prompt)

    print("\n\n[AI 주식 분석 리포트]")
    print(response.content)
    print("\n")

    # 이 노드는 최종 출력을 담당하므로 state를 추가로 업데이트하지 않음
    return

# ⛓️ Agent 워크플로우 그래프 생성 및 실행

앞서 정의한 **노드(Node)** 함수들을 **LangGraph**를 사용하여 연결하고, 실행 가능한 **Agent**를 완성합니다.<br>
전체 과정은 **그래프 정의 → 컴파일 및 시각화 → 실행**의 3단계로 진행됩니다.

<br>

### 1\. 워크플로우 정의

먼저 `StateGraph` 객체를 생성하고, `add_node`를 이용해 각 작업 함수를 노드로 추가합니다.<br>
그 후, `set_entry_point`로 시작 노드를 지정하고, `add_edge`로 노드들을 **주가 검색 → 뉴스 검색 → 분석** 순서로 연결하여 순차적인 워크플로우를 만듭니다.

> ```python
> # 노드 실행 순서를 정의합니다.
> workflow.set_entry_point("stock_price_search")
> workflow.add_edge("stock_price_search", "news_search")
> workflow.add_edge("news_search", "analysis")
> workflow.add_edge("analysis", END)
> ```

<br>

### 2\. 그래프 컴파일

정의된 워크플로우를 `compile()` 메서드를 통해 실행 가능한 애플리케이션(`app`)으로 만듭니다.

> ```python
> # 그래프를 실행 가능한 app으로 컴파일합니다.
> app = workflow.compile()
> ```

<br>

### 3\. Agent 실행 및 결과 확인

컴파일된 `app`에 분석할 회사 이름을 입력하여 Agent를 실행합니다.<br>
`app.stream()`을 사용하면 각 노드가 실행될 때마다 중간 결과와 상태 변화를 실시간으로 확인할 수 있어, Agent의 작동 과정을 단계별로 추적하는 데 유용합니다.

> ```python
> # app.stream()을 통해 각 단계의 실행 과정을 실시간으로 확인합니다.
> for event in app.stream(inputs):
>     ...
> ```

👇 아래 버튼을 눌러 코드를 실행합니다.

In [25]:
## 1. 워크플로우 정의
# 그래프의 상태(State)는 AgentState로 정의합니다.
workflow = StateGraph(AgentState)

# 1) 정의한 함수들을 노드로 추가합니다.
workflow.add_node("stock_price_search", stock_price_search_node)
workflow.add_node("news_search", news_search_node)
workflow.add_node("analysis", analysis_node)

# 2) 노드 실행 순서를 정의합니다. (A -> B -> C -> 종료)
workflow.set_entry_point("stock_price_search") # 시작점
workflow.add_edge("stock_price_search", "news_search")
workflow.add_edge("news_search", "analysis")
workflow.add_edge("analysis", END) # 'analysis' 노드 실행 후 워크플로우를 종료합니다.

## 2. 그래프 컴파일 및 시각화
# 1) 그래프를 실행 가능한 app으로 컴파일합니다.
app = workflow.compile()

## 2) 공통 함수를 사용하여 그래프를 시각화합니다.
display_mermaid_graph(app, title="순차적 주식 분석 워크플로우")

## 3. Agent 실행 및 결과 확인
# 1) Agent에게 전달할 초기 입력값입니다.
inputs = {"company_name": "삼성전자"}

# 2) app.stream()을 통해 각 단계의 실행 과정을 실시간으로 확인합니다.
for event in app.stream(inputs):
    pass


📊 [ 순차적 주식 분석 워크플로우 ]



--- (1/3) 주가 정보 검색 중... ---
--- (2/3) 관련 뉴스 검색 중... ---
--- (3/3) 종합 분석 리포트 생성 중... ---


[AI 주식 분석 리포트]
## 삼성전자 (005930) 투자 분석 리포트

**[개요]**

삼성전자 (005930)는 현재 107,500원에 거래되고 있으며, 배당수익률은 1.38%입니다. 최근 주가는 상승세를 보이며, 12개월 목표주가는 112,220원으로 제시되고 있습니다. 긍정적인 애널리스트 의견이 다수 존재하며, 2026년 1월 28일에 다음 실적 발표가 예정되어 있습니다. 최근 뉴스에서는 엔비디아와의 협력을 통한 '반도체 AI 팩토리' 구축, 3분기 실적 발표, 14년 연속 '동반성장지수' 최우수 획득 등 다양한 소식이 전해지고 있습니다.

**[긍정적 요인]**

*   **AI 반도체 시장 성장 기대**: 삼성전자는 엔비디아와의 협력을 통해 '반도체 AI 팩토리'를 구축하며, AI 시장 성장에 발맞춰 관련 기술 경쟁력을 강화하고 있습니다. 이는 메모리 반도체 및 시스템 반도체 분야에서 긍정적인 영향을 미칠 것으로 예상됩니다.
*   **견조한 실적**: 삼성전자는 2025년 3분기 실적 발표를 앞두고 있으며, 시장의 기대에 부응하는 실적을 발표할 경우 주가 상승의 모멘텀으로 작용할 수 있습니다.
*   **지속적인 사회적 책임**: 14년 연속 '동반성장지수' 최우수 획득은 삼성전자의 지속 가능한 성장에 대한 긍정적인 신호로 해석될 수 있으며, 투자자들의 신뢰를 높이는 요인으로 작용할 수 있습니다.
*   **긍정적인 애널리스트 전망**: 다수의 애널리스트가 삼성전자에 대해 매수 의견을 제시하고 있으며, 목표주가 또한 현재 주가보다 높게 형성되어 있어, 향후 주가 상승 가능성을 시사합니다.

**[부정적 요인]**

*   **글로벌 경제 불확실성**: 금리 인상, 인플레이션 등 글로벌 경제의 불확실성은 반도체 수요 감소 및 투자 심리 위축으로 이어져 주가에 부정적인 영향을 미칠 수 

# 📜 MCP 서버 스크립트 정의 (`search_server.py`)

이 스크립트는 MCP(Model Context Protocol)를 사용하여 금융 정보 검색 도구(Tool)들을 마이크로서비스 형태로 제공하는 **Python 서버**입니다.<br>
`search_server.py`라는 파일로 저장되며, 독립적으로 실행하여 다른 애플리케이션이나 Agent가 네트워크를 통해 이 도구들을 호출할 수 있도록 합니다.

<br>

#### 📝 제공하는 핵심 도구들

서버가 제공하는 핵심 도구는 다음과 같으며, 모두 LangChain의 `@tool` 데코레이터를 사용하여 표준화되었습니다.

  * **`get_stock_price`** : 특정 회사의 현재 주가를 검색합니다.
  * **`get_latest_news`** : 회사와 관련된 최신 뉴스를 검색합니다. `topic`을 지정하여 검색 주제를 좁힐 수 있습니다.
  * **`get_company_overview`** : 회사가 어떤 사업을 하는지 등 전반적인 개요를 검색합니다.
  * **`get_analyst_opinions`** : 해당 회사에 대한 증권사 애널리스트들의 최신 투자 의견을 검색합니다.

<br>

#### ⚙️ 서버 구성 방식

정의된 LangChain 도구들을 `langchain-mcp-adapters`를 통해 MCP 사양을 따르는 `FastMCP` 서버에 탑재합니다.<br>
스크립트가 직접 실행되면(`if __name__ == "__main__":`), `streamable-http` 전송 방식으로 서버가 구동됩니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [26]:
%%writefile /content/search_server.py
# ==================================
# Part 1: 라이브러리 임포트
# ==================================
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_mcp_adapters.tools import to_fastmcp
from mcp.server.fastmcp import FastMCP
from typing import Optional

# ==================================
# Part 2: 핵심 도구 초기화 및 정의
# ==================================
# Tavily 검색 엔진 초기화 (최대 1개 결과만 사용)
tavily_search = TavilySearchResults(max_results=1)

@tool
def get_stock_price(company_name: str) -> str:
    """
    특정 회사의 현재 실시간 주가를 검색합니다.
    '삼성전자'와 같이 회사 이름만 정확히 입력해야 합니다.
    """
    print(f"Executing get_stock_price for: {company_name}")
    return tavily_search.invoke({"query": f"{company_name} 현재 주가"})

@tool
def get_latest_news(company_name: str, topic: Optional[str] = None) -> str:
    """
    특정 회사와 관련된 최신 뉴스를 검색합니다.
    더 구체적인 뉴스를 원할 경우 '실적', '신제품' 등 관련 주제(topic)를 지정할 수 있습니다.
    """
    search_query = f"{company_name} {topic or '최신'} 뉴스"
    print(f"Executing get_latest_news with query: {search_query}")
    return tavily_search.invoke({"query": search_query})

@tool
def get_company_overview(company_name: str) -> str:
    """
    특정 회사가 어떤 사업을 하는지, 주요 제품은 무엇인지 등 사업의 전반적인 개요를 검색합니다.
    """
    print(f"Executing get_company_overview for: {company_name}")
    return tavily_search.invoke({"query": f"{company_name} 사업 개요 what they do"})

@tool
def get_analyst_opinions(company_name: str) -> str:
    """
    특정 회사에 대한 증권사 애널리스트들의 최신 리포트나 투자 의견을 검색합니다.
    """
    print(f"Executing get_analyst_opinions for: {company_name}")
    return tavily_search.invoke({"query": f"{company_name} 증권사 애널리스트 리포트 요약"})

# ==================================
# Part 3: MCP 서버 설정 및 실행
# ==================================
# 정의된 모든 도구들을 리스트로 취합
all_tools = [
    get_stock_price,
    get_latest_news,
    get_company_overview,
    get_analyst_opinions,
]

# LangChain 도구를 FastMCP 형식으로 변환
fastmcp_tools = [to_fastmcp(t) for t in all_tools]

# FastMCP 서버 인스턴스 생성
mcp = FastMCP(
    "FinancialInfoProvider",  # 서비스 이름
    tools=fastmcp_tools,
)

# 스크립트가 직접 실행될 때 서버를 구동
if __name__ == "__main__":
    mcp.run(transport="streamable-http")


Overwriting /content/search_server.py


# 🚀 금융 정보 MCP 서버 시작

앞서 작성한 `search_server.py` 스크립트를 백그라운드에서 실행하여 **MCP 서버**를 시작합니다.<br>
이 서버는 Agent가 네트워크를 통해 금융 정보 도구를 호출할 수 있는 엔드포인트 역할을 합니다.


👇 아래 버튼을 눌러 코드를 실행합니다.

In [27]:
# --- 금융 정보 MCP 서버 시작 (Port 8000) ---
start_mcp_server(
    script_name="search_server.py",
    port=8000,
    log_file="search_server.log",
    service_name="금융 정보"
)

🚀 금융 정보 MCP 서버를 시작합니다... (Port: 8000)
⏱️  서버 시작 대기 중... (최대 30초)
------------------------------
✅ 금융 정보 MCP 서버가 성공적으로 시작되어 실행 중입니다.
   (로그 파일: /content/search_server.log)


'SUCCESS'

# 🤖 자율(ReAct) Agent 워크플로우

이번에는 이전의 순차적 그래프에서 더 나아가, **ReAct (Reasoning and Acting) 패턴**을 따르는 자율적인 Agent를 구축합니다.<br>
이 Agent는 **스스로 판단하여 도구를 반복적으로 호출**하고, 모든 정보가 수집되었다고 판단되면 최종 리포트를 생성하는 동적인 워크플로우를 가집니다.

<br>

### 1\. Agent 아키텍처

Agent는 다음과 같은 핵심 구성 요소로 이루어집니다.

  * **MCP 클라이언트** : 원격 서버에 접속하여 도구들을 불러옵니다.
  * **Agent 상태 (`AutonomousState`)** : 대화 기록(`messages`)과 실행 횟수(`turn_count`)를 저장합니다.
  * **핵심 노드 (Nodes)** : `agent_node` (판단), `tool_node` (실행), `report_node` (보고)로 역할을 나눕니다.
  * **조건부 라우터 (`agent_router`)** : `agent_node`의 판단에 따라 다음 작업으로 동적으로 분기합니다.

<br>

### 2\. Agent 상태(`AutonomousState`) 정의

Agent의 대화 기록(`messages`)과 함께, 무한 루프를 방지하기 위해 실행 횟수를 기록하는 `turn_count`를 상태에 추가합니다.

> ```python
> class AutonomousState(TypedDict):
>     # 대화 기록이 계속 추가되도록 설정
>     messages: Annotated[list[BaseMessage], operator.add]
>     # Agent의 판단(turn) 횟수 기록
>     turn_count: int
> ```

<br>

### 3\. Agent 그래프 생성 (`create_financial_agent_graph`)

Agent를 생성하는 전체 로직을 하나의 비동기 함수(`async def`) 안에 캡슐화하여 코드의 재사용성과 가독성을 높입니다.

> ```python
> async def create_financial_agent_graph():
>     # 1. 클라이언트 설정 및 모델 준비
>     # 2. 핵심 노드 함수 정의
>     # 3. 조건부 라우터 함수 정의
>     # 4. 그래프 구성 및 컴파일
>     return workflow.compile()
> ```

> #### 3.1. 클라이언트 및 모델 준비
>
> MCP 서버에 접속하여 **원격 도구들을 로드**합니다.<br>
> 또한, **도구 호출을 위한 Agent용 모델**과 **리포트 작성을 위한 Reporter용 모델**을 별도로 정의하여 역할을 분리합니다.
>
> > ```python
> > # MCP 서버에 접속하여 도구 로드
> > client = MultiServerMCPClient(...)
> > tools = await client.get_tools()
> >
> > # 역할에 따라 두 개의 모델 준비
> > agent_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0).bind_tools(tools)
> > reporting_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
> > ```

> #### 3.2. 핵심 노드 정의
>
> `agent_node`(판단), `tool_node`(실행), `report_node`(보고) 세 가지 핵심 노드 함수를 정의합니다.
>
> > ```python
> > def agent_node(state: AutonomousState):
> >     # ... LLM을 호출하여 다음 행동 결정 ...
> >     return {"messages": [response], "turn_count": ...}
> >
> > # ToolNode는 LangGraph에서 제공하는 기본 도구 실행 노드
> > tool_node = ToolNode(tools)
> >
> > def report_node(state: AutonomousState):
> >     # ... 수집된 정보로 최종 리포트 작성 ...
> >     return {"messages": [response]}
> > ```

> #### 3.3. 조건부 라우터(Router) 및 그래프 구성
>
> **`agent_router`** 함수는 Agent의 마지막 메시지를 보고 **도구를 추가로 호출할지, 아니면 리포트 생성을 시작할지** 동적으로 결정합니다.<br>
> 이 라우터를 `add_conditional_edges`에 연결하여 그래프의 흐름을 제어합니다.
>
> > ```python
> > def agent_router(state: AutonomousState):
> >     # ... 최대 반복 횟수 체크 (안전장치) ...
> >     if state["messages"][-1].tool_calls:
> >         return "call_tool"  # 도구 호출
> >     else:
> >         return "generate_report" # 리포트 생성
> >
> > # 그래프 구성
> > workflow = StateGraph(AutonomousState)
> > workflow.add_node("financial_analysis_agent", agent_node)
> > workflow.add_node("search_tools", ToolNode(tools))
> > workflow.add_node("reporter", report_node)
> >
> > # agent 노드의 결과에 따라 동적으로 분기하도록 설정
> > workflow.add_conditional_edges("financial_analysis_agent", agent_router, {
> >     "call_tool": "search_tools",
> >     "generate_report": "reporter"
> > })
> > ```

<br>

> ### 4\. Agent 실행 및 시각화 (`main`)
>
> `main` 함수에서 생성된 그래프(`app`)를 **실행하고, 그 구조를 시각화**합니다.<br>
> `app.astream()`을 사용하여 Agent의 각 단계를 실시간으로 관찰합니다.
>
> > ```python
> > async def main():
> >     # 1. 그래프 생성 함수를 호출하여 app을 받음
> >     app = await create_financial_agent_graph()
> >
> >     # 2. 그래프 구조를 다이어그램으로 시각화
> >     display_mermaid_graph(app, title="금융 정보 ReAct Agent")
> >
> >     # 3. 입력을 정의하고 Agent 실행
> >     inputs = {"messages": [HumanMessage(content="...")]}
> >     async for update in app.astream(inputs):
> >         # ... 각 단계의 결과 출력 ...
> >
> > # 메인 함수 실행
> > await main()
> > ```


👇 아래 버튼을 눌러 코드를 실행합니다.

In [28]:
# --- 1. 필요한 모든 모듈 임포트 ---
import asyncio
import operator
import textwrap
from typing import List, TypedDict, Annotated

from IPython.display import HTML, display
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


# --- 2. Agent의 상태(State) 정의 ---
class AutonomousState(TypedDict):
    """
    자율 Agent의 상태를 정의합니다.
    - messages: 대화 기록이 누적됩니다.
    - turn_count: Agent의 실행 횟수를 기록하여 무한 루프를 방지합니다.
    """
    messages: Annotated[List[BaseMessage], operator.add]
    turn_count: int


# --- 3. 금융 정보 Agent 그래프 생성 함수 ---
async def create_financial_agent_graph():
    """
    금융 정보 분석 ReAct Agent의 실행 가능한 그래프(app)를 생성하고 반환합니다.
    이 함수는 클라이언트 설정, 모델 준비, 노드 정의, 그래프 구성을 모두 포함합니다.
    """
    # --- 3-1. MCP 클라이언트 설정 및 도구/모델 준비 ---
    print("📡 MCP 클라이언트를 HTTP 방식으로 설정합니다...")
    client = MultiServerMCPClient(
        {"financial_service": {"transport": "streamable_http", "url": "http://localhost:8000/mcp/"}}
    )
    tools = await client.get_tools()
    print(f"✅ 로딩된 도구: {[tool.name for tool in tools]}")

    # 도구 호출에 특화된 모델
    agent_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0).bind_tools(tools)
    # 리포트 생성에 특화된 모델
    reporting_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

    # --- 3-2. 핵심 노드 함수 정의 ---
    def agent_node(state: AutonomousState):
        """사용자 요청과 이전 기록을 바탕으로 다음 행동을 결정하는 노드"""
        current_turn = state.get("turn_count", 0)
        print(f"\n++++++++++ 🤖 FINANCIAL AGENT (Turn {current_turn + 1}) ++++++++++")
        print("사용자 요청과 이전 기록을 바탕으로 다음 행동을 결정합니다...")
        system_prompt = SystemMessage(
            content="""당신은 사용자의 요청을 해결하기 위해 필요한 도구를 호출하는 역할만 수행합니다.
            절대로 중간에 답변이나 요약을 생성하지 마세요.
            오직 다음으로 호출할 단 하나의 도구를 결정하고 tool_calls를 생성하세요.
            만약 모든 정보가 수집되었다고 생각되면, tool_calls 없이 '모든 정보 수집 완료'라고만 답변하세요.
            """
        )
        messages_with_system_prompt = [system_prompt, *state["messages"]]
        response = agent_model.invoke(messages_with_system_prompt)
        return {"messages": [response], "turn_count": current_turn + 1}

    def report_node(state: AutonomousState):
        """수집된 모든 정보를 종합하여 최종 리포트를 작성하는 노드"""
        print("\n++++++++++ ✍️ FINANCIAL REPORTER ++++++++++")
        print("수집된 모든 정보를 종합하여 최종 리포트를 작성합니다...")
        report_history = [msg for msg in state["messages"] if not isinstance(msg, HumanMessage)]
        search_history = textwrap.dedent("\n".join([msg.pretty_repr() for msg in report_history]))
        report_prompt = f"""당신은 전문 애널리스트입니다.
        아래 정보 수집 기록을 바탕으로 사용자의 초기 질문에 대한 최종 투자 리포트를 작성해주세요.
        \n\n[정보 수집 기록]
        \n{search_history}
        """
        response = reporting_model.invoke(report_prompt)
        return {"messages": [response]}

    # --- 3-3. 조건부 엣지(Edge) 함수 정의 ---
    MAX_TURNS = 5
    def agent_router(state: AutonomousState):
        """Agent의 응답에 따라 다음 경로를 결정하는 라우터"""
        print("\n----- 🚦 ROUTER -----")
        if state.get("turn_count", 0) >= MAX_TURNS:
            print(f"⚠️ Safety Guard: 최대 반복 횟수({MAX_TURNS}회)를 초과하여 강제로 리포트를 생성합니다.\n")
            return "generate_report"

        last_message = state["messages"][-1]
        if last_message.tool_calls:
            print("분기: [call_tool] -> TOOLS 노드로 이동합니다.\n")
            return "call_tool"
        else:
            print("분기: [generate_report] -> REPORTER 노드로 이동합니다.\n")
            return "generate_report"

    # --- 3-4. 그래프 구성 및 컴파일 ---
    workflow = StateGraph(AutonomousState)
    workflow.add_node("financial_analysis_agent", agent_node)
    workflow.add_node("search_tools", ToolNode(tools))
    workflow.add_node("reporter", report_node)

    workflow.set_entry_point("financial_analysis_agent")
    workflow.add_conditional_edges("financial_analysis_agent", agent_router, {"call_tool": "search_tools", "generate_report": "reporter"})
    workflow.add_edge("search_tools", "financial_analysis_agent")
    workflow.add_edge("reporter", END)

    return workflow.compile()


# --- 4. 메인 실행 함수 ---
async def main():
    """그래프 생성, 시각화, 실행을 총괄하는 비동기 메인 함수"""
    # --- 4-1. 그래프 생성 함수 호출 ---
    app = await create_financial_agent_graph()
    print("✅ 그래프가 성공적으로 컴파일되었습니다.")

    # --- 4-2. 시각화 ---
    display_mermaid_graph(app, title="금융 정보 ReAct Agent")

    # --- 4-3. Agent 실행 ---
    print("\n================ ReAct Agent 모델 실행 ================")
    inputs = {"messages": [HumanMessage(content="""
    엔비디아(NVIDIA)에 대한 종합 투자 리포트를 작성해줘. 최신 주가, 사업 개요, 그리고 'AI 칩'과 관련된 최신 뉴스를 포함해서 분석해줘.
    """)], "turn_count": 0}

    # 사용자 입력을 먼저 출력
    print("\n++++++++++ 🙋‍♂️ USER INPUT ++++++++++")
    inputs["messages"][0].pretty_print()

    # stream_mode="updates"를 사용하여 각 노드의 출력을 순서대로 처리
    async for update in app.astream(inputs, stream_mode="updates"):
        # update는 {'node_name': {'key': 'value'}} 형태의 딕셔너리
        node_name = list(update.keys())[0]
        node_output = list(update.values())[0]

        if node_name == "financial_analysis_agent":
            # agent 노드의 AiMessage를 출력
            node_output['messages'][-1].pretty_print()
        elif node_name == "search_tools":
            # tools 노드는 ToolMessage 리스트를 반환
            for tool_message in node_output['messages']:
                tool_message.pretty_print()
        elif node_name == "reporter":
            # reporter 노드의 최종 결과 AiMessage를 출력
            node_output['messages'][-1].pretty_print()

    print("\n✅ 에이전트 실행이 완료되었습니다.")


# --- 5. 비동기 메인 함수 실행 ---
await main()

📡 MCP 클라이언트를 HTTP 방식으로 설정합니다...
✅ 로딩된 도구: ['get_stock_price', 'get_latest_news', 'get_company_overview', 'get_analyst_opinions']
✅ 그래프가 성공적으로 컴파일되었습니다.

📊 [ 금융 정보 ReAct Agent ]




================ ReAct Agent 모델 실행 ================

++++++++++ 🙋‍♂️ USER INPUT ++++++++++
================================ Human Message =================================


    엔비디아(NVIDIA)에 대한 종합 투자 리포트를 작성해줘. 최신 주가, 사업 개요, 그리고 'AI 칩'과 관련된 최신 뉴스를 포함해서 분석해줘.
    

++++++++++ 🤖 FINANCIAL AGENT (Turn 1) ++++++++++
사용자 요청과 이전 기록을 바탕으로 다음 행동을 결정합니다...

----- 🚦 ROUTER -----
분기: [call_tool] -> TOOLS 노드로 이동합니다.

================================== Ai Message ==================================
Tool Calls:
  get_stock_price (6bc35d36-eba8-44ec-80c9-c75a48525a10)
 Call ID: 6bc35d36-eba8-44ec-80c9-c75a48525a10
  Args:
    company_name: NVIDIA
  get_company_overview (97831cea-8cb1-43bf-a773-6b4dbc26de4e)
 Call ID: 97831cea-8cb1-43bf-a773-6b4dbc26de4e
  Args:
    company_name: NVIDIA
  get_latest_news (1e6f1d06-5a87-4028-9135-8c957ad62a72)
 Call ID: 1e6f1d06-5a87-4028-9135-8c957ad62a72
  Args:
    topic: AI 칩
    company_name: NVIDIA
================================= Tool Message ================

# 🛑 금융 정보 MCP 서버 종료

모든 작업이 완료된 후, 백그라운드에서 실행되던 **MCP 서버 프로세스를 종료**하여 시스템 자원을 정리합니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [29]:
# --- 금융 정보 MCP 서버 종료 (Port 8000) ---
stop_mcp_server(port=8000, service_name="금융 정보")


🛑 금융 정보 MCP 서버(Port: 8000)를 종료합니다...
✅ 금융 정보 서버가 종료되었습니다.


# 📜 포트폴리오 MCP 서버 스크립트 (db_server.py)

이 스크립트는 **SQLite 데이터베이스와 상호작용**하는 도구들을 제공하는 **두 번째 MCP 서버**를 생성합니다.<br>
`db_server.py` 파일로 저장되며, Agent가 주식 포트폴리오 데이터를 조회하고 분석할 수 있도록 SQL 쿼리 관련 기능들을 API 형태로 노출하는 역할을 합니다.

### 1\. 주요 기능 및 아키텍처

  * **데이터베이스 생성**: 스크립트 실행 시, `my_portfolio`라는 테이블을 가진 예제용 SQLite 데이터베이스(`portfolio.db`)를 자동으로 생성하고 샘플 데이터를 삽입합니다.

    ```sql
    CREATE TABLE my_portfolio (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ticker TEXT NOT NULL UNIQUE,
        company_name TEXT NOT NULL,
        shares_owned INTEGER NOT NULL,
        purchase_price REAL NOT NULL
    )
    ```

  * **`SQLDatabaseToolkit` 활용**: 이 스크립트의 핵심은 LangChain의 `SQLDatabaseToolkit`입니다.<br>
  이 툴킷은 데이터베이스 연결 정보(`db`)와 LLM만 제공하면, 해당 데이터베이스와 상호작용하는 데 필요한 표준 도구들을 LLM에게 제공합니다.

  * **자동 생성 도구 목록**: `SQLDatabaseToolkit`은 다음과 같은 유용한 도구들을 만들어냅니다.

      * **`sql_db_list_tables`** : 데이터베이스 내의 모든 테이블 목록을 조회합니다.
      * **`sql_db_schema`** : 특정 테이블의 스키마(구조) 정보를 보여줍니다.
      * **`sql_db_query`** : 실제 SQL 쿼리를 실행하고 결과를 가져옵니다.
      * **`sql_db_query_checker`** : 실행하기 전에 SQL 쿼리에 문법적 오류가 없는지 확인합니다.

  * **서버 실행**: 자동으로 생성된 DB 도구들을 `FastMCP` 서버에 탑재하여 **포트 8001**에서 실행합니다. (기존 서버와의 충돌 방지)

    ```python
    if __name__ == "__main__":
        # 1. DB 생성 및 초기화
        setup_portfolio_database()
        # 2. MCP 서버 실행
        mcp.run(transport="streamable-http")
    ```

### 2\. `SQLDatabaseToolkit` 사용법

데이터베이스 경로를 지정하여 `SQLDatabase` 객체를 만들고, 이를 `SQLDatabaseToolkit`에 `db`와 `llm` 인자와 함께 전달하면 됩니다.

> ```python
> # 데이터베이스 연결
> db = SQLDatabase.from_uri("sqlite:///portfolio.db")
>
> # LLM과 DB 연결 정보를 제공하여 툴킷 생성
> toolkit = SQLDatabaseToolkit(db=db, llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash"))
>
> # 툴킷에서 자동으로 생성된 도구 목록 가져오기
> db_tools = toolkit.get_tools()
> ```

👇 아래 버튼을 눌러 코드를 실행합니다.

In [30]:
%%writefile /content/db_server.py
# SQLite 데이터베이스와 상호작용하는 도구를 제공하는 MCP 서버입니다.

# --- 1. 라이브러리 임포트 ---
import sqlite3
import uvicorn
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_mcp_adapters.tools import to_fastmcp
from mcp.server.fastmcp import FastMCP

# --- 2. 데이터베이스 설정 및 생성 함수 ---
DB_PATH = "portfolio.db"

def setup_portfolio_database():
    """개인 포트폴리오 예제용 SQLite 데이터베이스를 생성하고 초기화합니다."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS my_portfolio")
    cursor.execute("""
    CREATE TABLE my_portfolio (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ticker TEXT NOT NULL UNIQUE,
        company_name TEXT NOT NULL,
        shares_owned INTEGER NOT NULL,
        purchase_price REAL NOT NULL
    )
    """)
    portfolio_data = [
        ('NVDA', 'NVIDIA Corp', 50, 120.50),
        ('AAPL', 'Apple Inc', 100, 185.75),
        ('TSLA', 'Tesla Inc', 30, 250.00)
    ]
    cursor.executemany(
        "INSERT INTO my_portfolio (ticker, company_name, shares_owned, purchase_price) VALUES (?, ?, ?, ?)",
        portfolio_data
    )
    conn.commit()
    conn.close()
    print(f"✅ 포트폴리오 데이터베이스 '{DB_PATH}'가 성공적으로 생성 및 초기화되었습니다.")


# --- 3. MCP 서버 설정 및 실행 ---
if __name__ == "__main__":

    # 1. (가장 중요) 스크립트 실행 시 DB부터 생성합니다.
    setup_portfolio_database()

    # 2. DB가 확실히 생성된 *이후에* DB 연결 객체를 생성합니다.
    db = SQLDatabase.from_uri(f"sqlite:///{DB_PATH}")

    # 3. SQL 툴킷을 생성합니다.
    toolkit = SQLDatabaseToolkit(db=db, llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash"))

    # 4. 툴킷에서 도구 목록을 가져옵니다.
    db_tools = toolkit.get_tools()
    print(f"🛠️  자동 생성된 DB 도구: {[tool.name for tool in db_tools]}")

    # 5. LangChain 도구를 MCP 서버가 이해할 수 있는 형식으로 변환합니다.
    fastmcp_tools = [to_fastmcp(t) for t in db_tools]

    # 6. MCP 서버 인스턴스를 생성합니다. (포트 8001 사용)
    mcp = FastMCP("PortfolioDBService", tools=fastmcp_tools, port=8001)

    # 7. streamable-http 방식으로 서버를 실행합니다.
    mcp.run(transport="streamable-http")

Overwriting /content/db_server.py


# 🚀 포트폴리오 MCP 서버 시작

앞서 작성한 `db_server.py` 스크립트를 백그라운드에서 실행하여 **데이터베이스 도구를 제공하는 두 번째 MCP 서버**를 시작합니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [31]:
# --- 포트폴리오 MCP 서버 시작 (Port 8001) ---
start_mcp_server(
    script_name="db_server.py",
    port=8001,
    log_file="db_server.log",
    service_name="포트폴리오 DB"
)

🚀 포트폴리오 DB MCP 서버를 시작합니다... (Port: 8001)
⏱️  서버 시작 대기 중... (최대 30초)
------------------------------
✅ 포트폴리오 DB MCP 서버가 성공적으로 시작되어 실행 중입니다.
   (로그 파일: /content/db_server.log)


'SUCCESS'

# 🗃️ 포트폴리오 DB 조회 Agent 워크플로우

마지막으로, **데이터베이스 조회에만 특화**된 자율 Agent를 구축합니다.<br>
이 Agent는 이전의 범용 Agent와 달리, 오직 DB 관련 도구만 사용하여 사용자의 질문에 답변하는 전문가 역할을 수행합니다.

<br>

### 1\. Agent 아키텍처 및 특징

  * **전용 클라이언트**: Agent는 금융 정보 서버가 아닌, DB 서버(포트 8001)에만 연결하는 전용 `MultiServerMCPClient`를 사용합니다.<br>
  이를 통해 Agent가 사용할 수 있는 도구를 DB 관련 기능(`sql_db_query` 등)으로 제한합니다.

    ```python
    client = MultiServerMCPClient({
        "database_service": {
            "transport": "streamable_http",
            "url": "http://localhost:8001/mcp/",
        }
    })
    db_tools = await client.get_tools()
    ```

  * **체계적 사고를 위한 시스템 프롬프트** : 이 Agent의 가장 큰 특징은 **명확한 행동 지침**을 담은 `SystemMessage`를 받는다는 점입니다. Agent는 이 지침에 따라 체계적으로 사고하고 행동합니다.

    ```python
    system_prompt = """
    당신은 ... 데이터베이스 전문가입니다.

    **행동 지침:**
    1. 가장 먼저 `sql_db_list_tables`를 호출하여 ... 테이블을 확인하세요.
    2. 그 다음, `sql_db_schema`를 호출하여 ... 구조를 파악하세요.
    3. 마지막으로, `sql_db_query`를 실행하여 ... 답을 찾으세요.
    """
    ```

  * **단순화된 그래프 구조** : 이전 Agent와 달리 별도의 `report_node`가 없습니다. `agent_node`가 도구 호출(Reasoning)과 최종 답변 생성을 모두 담당하여 구조가 더 단순해졌습니다.<br>
  라우터 역시 도구를 호출할지(`call_tool`), 아니면 프로세스를 종료할지(`end`)만 결정합니다.

    ```python
    # 라우터의 분기 로직
    def agent_router(state: AutonomousState):
        if state["messages"][-1].tool_calls:
            return "call_tool"
        else:
            return "end"

    # 그래프 구성
    workflow.add_conditional_edges("text_to_sql_agent", agent_router, {
        "call_tool": "db_tools",
        "end": END
    })
    ```

### 2\. 실행 및 결과 확인

`main` 함수에서 생성된 DB 조회 Agent를 실행하고, 사용자의 질문("내 포트폴리오의 모든 종목 티커와 보유 수량을 알려줘")에 대해 Agent가 어떻게 단계별로 DB 도구를 사용하여 답변을 찾아가는지 실시간으로 확인할 수 있습니다.


👇 아래 버튼을 눌러 코드를 실행합니다.

In [32]:
# --- 1. 필요한 모든 모듈 임포트 ---
import asyncio
import operator
import textwrap
from typing import List, TypedDict, Annotated

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from IPython.display import display, HTML


# --- 2. Agent의 상태(State) 정의 ---
class AutonomousState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    turn_count: int


# --- 3. DB 조회 Agent 그래프 생성 함수 ---
async def create_db_agent_graph():
    """
    DB 조회 ReAct Agent의 실행 가능한 그래프(app)를 생성하고 반환합니다.
    """
    # --- 3-1. DB 서버 클라이언트 설정 및 도구/모델 준비 ---
    print("📡 DB 서버에만 연결하는 MCP 클라이언트를 설정합니다...")
    client = MultiServerMCPClient(
        {"database_service": {"transport": "streamable_http", "url": "http://localhost:8001/mcp/"}}
    )
    db_tools = await client.get_tools()
    print(f"✅ 포트폴리오 Agent용 도구 로딩 완료: {[tool.name for tool in db_tools]}")

    agent_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0).bind_tools(db_tools)

    # --- 3-2. 핵심 노드 함수 정의 ---
    def agent_node(state: AutonomousState):
        current_turn = state.get("turn_count", 0)
        print(f"\n++++++++++ 🤖 PORTFOLIO AGENT (Turn {current_turn + 1}) ++++++++++")
        print("DB 조회를 위해 다음 행동을 결정합니다...")

        system_prompt = """
        당신은 사용자의 포트폴리오 데이터베이스를 조회하여 질문에 답변하는 데이터베이스 전문가입니다.

        **행동 지침:**
        1. 가장 먼저 `sql_db_list_tables`를 호출하여 사용 가능한 테이블을 확인하세요.
        2. 그 다음, `sql_db_schema`를 호출하여 관련된 테이블의 구조를 파악하세요.
        3. 마지막으로, 알아낸 테이블과 스키마 정보를 바탕으로 `sql_db_query`를 실행하여 질문에 대한 답을 찾으세요.
        """
        messages_with_system_prompt = [SystemMessage(content=system_prompt), *state["messages"]]

        response = agent_model.invoke(messages_with_system_prompt)
        # 도구 호출이 아닌, 최종 답변일 경우에만 제목을 추가합니다.
        if not response.tool_calls:
            response.content = "## 포트폴리오 조회 결과\n\n" + response.content

        return {"messages": [response], "turn_count": current_turn + 1}

    tool_node = ToolNode(db_tools)

    # --- 3-3. 조건부 엣지(Edge) 함수 정의 ---
    MAX_TURNS = 5
    def agent_router(state: AutonomousState):
        print("\n----- 🚦 ROUTER -----")
        if state.get("turn_count", 0) >= MAX_TURNS:
            print(f"⚠️ Safety Guard: 최대 반복 횟수({MAX_TURNS}회)를 초과하여 강제로 종료합니다.\n")
            return "end"

        last_message = state["messages"][-1]
        if last_message.tool_calls:
            print("분기: [call_tool] -> TOOLS 노드로 이동합니다.\n")
            return "call_tool"
        else:
            print("분기: [end] -> 최종 답변이 생성되어 종료합니다.\n")
            return "end"

    # --- 3-4. 그래프 구성 및 컴파일 ---
    workflow = StateGraph(AutonomousState)
    workflow.add_node("text_to_sql_agent", agent_node)
    workflow.add_node("db_tools", tool_node)

    workflow.set_entry_point("text_to_sql_agent")
    workflow.add_conditional_edges("text_to_sql_agent", agent_router, {"call_tool": "db_tools", "end": END})
    workflow.add_edge("db_tools", "text_to_sql_agent")

    return workflow.compile()


# --- 4. 메인 실행 함수 ---
async def main():
    # --- 4-1. 그래프 생성 함수 호출 ---
    app = await create_db_agent_graph()
    print("✅ 포트폴리오 Agent 그래프가 성공적으로 컴파일되었습니다.")

    # --- 4-2. 시각화 ---
    display_mermaid_graph(app, title="포트폴리오 DB 조회 Agent")

    # --- 4-3. Agent 실행 ---
    print("\n================ PortfolioAgent 테스트 실행 ================")
    inputs = {"messages": [HumanMessage(content="내 포트폴리오에 있는 모든 종목의 티커와 보유 수량을 알려줘.")], "turn_count": 0}

    async for event in app.astream(inputs, stream_mode="values"):
        last_message = event["messages"][-1]
        if isinstance(last_message, HumanMessage):
            print("\n++++++++++ 🙋‍♂️ USER INPUT ++++++++++")
            last_message.pretty_print()
        else:
            last_message.pretty_print()

# --- 5. 비동기 메인 함수 실행 ---
await main()

📡 DB 서버에만 연결하는 MCP 클라이언트를 설정합니다...
✅ 포트폴리오 Agent용 도구 로딩 완료: ['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']
✅ 포트폴리오 Agent 그래프가 성공적으로 컴파일되었습니다.

📊 [ 포트폴리오 DB 조회 Agent ]




================ PortfolioAgent 테스트 실행 ================

++++++++++ 🙋‍♂️ USER INPUT ++++++++++
================================ Human Message =================================

내 포트폴리오에 있는 모든 종목의 티커와 보유 수량을 알려줘.

++++++++++ 🤖 PORTFOLIO AGENT (Turn 1) ++++++++++
DB 조회를 위해 다음 행동을 결정합니다...

----- 🚦 ROUTER -----
분기: [call_tool] -> TOOLS 노드로 이동합니다.

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (775ef8d6-0451-4897-a2e0-4c3ce432ad1d)
 Call ID: 775ef8d6-0451-4897-a2e0-4c3ce432ad1d
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

my_portfolio

++++++++++ 🤖 PORTFOLIO AGENT (Turn 2) ++++++++++
DB 조회를 위해 다음 행동을 결정합니다...

----- 🚦 ROUTER -----
분기: [call_tool] -> TOOLS 노드로 이동합니다.

================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (c62856bd-9c35-486b-973a-f0d285be65ad)
 Call ID: c62856bd-9c35-

# 🛑 포트폴리오 MCP 서버 종료

모든 작업이 완료되었으므로, 백그라운드에서 실행되던 **DB 서버(포트 8001)** 프로세스를 종료하여 시스템 자원을 정리합니다.


👇 아래 버튼을 눌러 코드를 실행합니다.

In [33]:
# --- 포트폴리오 MCP 서버 종료 (Port 8001) ---
stop_mcp_server(port=8001, service_name="포트폴리오 DB")


🛑 포트폴리오 DB MCP 서버(Port: 8001)를 종료합니다...
✅ 포트폴리오 DB 서버가 종료되었습니다.


# 👑 Orchestrator: 전문가 Agent들을 지휘하는 상위 Agent

지금까지 만든 `금융 정보 Agent`와 `포트폴리오 DB 조회 Agent`를 각각의 전문가로 보고, 이 전문가들을 총괄하여 복잡한 임무를 수행하게 할 `상위 Agent(Orchestrator)`를 구축합니다.<br>
이는 **단일 Agent로는 해결하기 어려운 복잡한 문제를 여러 개의 단순한 하위 문제로 나누어 해결**할 수 있게 해주는 매우 강력한 계층적 접근 방식입니다.

<br>

### 1. 하위 Agent를 '도구(Tool)'로 추상화

> 가장 핵심적인 아이디어는, 앞서 만든 두 전문가 Agent를 **하나의 '도구(함수)'처럼** 만드는 것입니다.<br>
상위 Agent가 이 도구를 호출하면, 내부적으로는 해당 하위 Agent의 전체 워크플로우가 실행되고 그 최종 결과만 반환됩니다.
>
> ```python
> @tool
> async def financial_info_agent_tool(query: str) -> str:
>     """금융 정보 분석 전문가 Agent를 호출하는 도구"""
>     # 1. 하위 Agent 그래프 생성
>     app = await create_financial_agent_graph()
>     # 2. 하위 Agent 실행 및 결과 반환
>     response = await app.ainvoke({"messages": [HumanMessage(content=query)], ...})
>     return response['messages'][-1].content
> ```

### 2. 상위 Agent (Orchestrator) 워크플로우

> 상위 Agent는 '계획을 세우는 Agent'와 '최종 보고서를 작성하는 Reporter'의 역할로 나뉩니다.
>
> #### 2.1. 마스터 Agent (`orchestrator_agent_node`)
>
> > 사용자의 복잡한 질문을 받고, **어떤 전문가 Agent(도구)를 어떤 순서로 호출할지 계획**을 세우는 '마스터 Agent' 역할을 합니다. **한 번에 하나의 도구만 호출**하도록 규칙을 설정하여 신중하게 작업을 수행합니다.
> >
> > ```python
> > system_prompt = """
> > 당신은 전문가 팀을 이끄는 마스터 Agent입니다. 당신의 임무는 사용자의 복잡한 요청을 해결하기 위해, 가장 적합한 전문가 Agent(도구)를 순서대로 호출하는 것입니다.
> > **매우 중요한 규칙:**
> > 1. 당신은 도구를 반드시 **한 번에 하나씩만** 순서대로 호출해야 합니다.
> > 2. **모든 정보 수집이 완료되었다고 판단되면, ... '모든 정보 수집 완료'라고만 답변하세요.**
> > """
> > ```
>
> #### 2.2. 최고 투자 책임자 (`orchestrator_report_node`)
>
> > 모든 하위 Agent들의 작업이 끝나면, 각 전문가가 가져온 정보들을 모두 모아 **'최고 투자 책임자(CIO)'의 관점에서 최종 종합 리포트를 작성**합니다.
> >
> > ```python
> > report_prompt = f"""
> > 당신은 최고 투자 책임자(CIO)입니다. 아래는 각 팀(하위 Agent)이 수집하고 분석한 투자 정보입니다.
> >
> > [각 팀의 분석 보고 내용]
> > {dedented_history}
> >
> > ... 최종 종합 투자 리포트를 상세하고 명확하게 작성해주세요.
> > """
> > ```

### 3. 실행 및 결과 확인

> 복잡한 사용자 질문("내 포트폴리오의 모든 종목에 대해 최신 주가와 부정적 뉴스를 종합해서 리포트 해줘")이 입력되면, 상위 Agent는 다음과 같이 행동합니다.
>
> 1.  **포트폴리오 DB 조회 Agent**를 호출하여 포트폴리오에 어떤 종목이 있는지 파악합니다.
> 2.  알아낸 각 종목에 대해 **금융 정보 Agent**를 순서대로 호출하여 주가와 뉴스를 분석합니다.
> 3.  모든 정보가 취합되면, 최종 리포트 노드가 **종합적인 투자 리포트를 생성**하며 프로세스를 완료합니다.

👇 아래 버튼을 눌러 코드를 실행합니다.

In [34]:
# --- 1. 금융 정보 MCP 서버 시작 및 확인 (Port 8000) ---
start_mcp_server(
    script_name="search_server.py",
    port=8000,
    log_file="search_server.log",
    service_name="금융 정보"
)

print("\n" + "="*40 + "\n")

# --- 2. 포트폴리오 MCP 서버 시작 및 확인 (Port 8001) ---
start_mcp_server(
    script_name="db_server.py",
    port=8001,
    log_file="db_server.log",
    service_name="포트폴리오 DB"
)

print("\n" + "="*40)
print("✅ 모든 MCP 서버 시작 작업이 완료되었습니다.")
print("   각 서버의 최종 상태를 위 로그에서 확인하세요.")

🚀 금융 정보 MCP 서버를 시작합니다... (Port: 8000)
⏱️  서버 시작 대기 중... (최대 30초)
------------------------------
✅ 금융 정보 MCP 서버가 성공적으로 시작되어 실행 중입니다.
   (로그 파일: /content/search_server.log)


🚀 포트폴리오 DB MCP 서버를 시작합니다... (Port: 8001)
⏱️  서버 시작 대기 중... (최대 30초)
------------------------------
✅ 포트폴리오 DB MCP 서버가 성공적으로 시작되어 실행 중입니다.
   (로그 파일: /content/db_server.log)

✅ 모든 MCP 서버 시작 작업이 완료되었습니다.
   각 서버의 최종 상태를 위 로그에서 확인하세요.


👇 아래 버튼을 눌러 코드를 실행합니다.

In [35]:
# --- 1. 필요한 모든 모듈 임포트 ---
import asyncio
import operator
import textwrap
from typing import List, TypedDict, Annotated

from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


# --- 2. 하위 Agent를 호출하는 '도구(Tool)' 함수 정의 ---
@tool
async def financial_info_agent_tool(query: str) -> str:
    """
    특정 회사에 대한 최신 주가, 뉴스, 사업 개요 등 금융 정보를 상세히 분석하고 리포트를 작성할 때 사용합니다.
    예: '삼성전자 주가와 최신 AI 관련 뉴스 알려줘'
    """
    print(f"\n>>>> [호출] 금융 정보 Agent (입력: {query})")
    app = await create_financial_agent_graph() # 이전 단계에서 정의한 금융 정보 조회 에이전트를 재사용합니다.
    response = await app.ainvoke({"messages": [HumanMessage(content=query)], "turn_count": 0})
    print("<<<< [완료] 금융 정보 Agent")
    return response["messages"][-1].content

@tool
async def portfolio_db_agent_tool(query: str) -> str:
    """
    사용자의 개인 주식 포트폴리오 데이터베이스를 조회할 때 사용합니다.
    보유 종목, 수량, 매수 가격 등의 정보를 확인할 수 있습니다.
    예: '내 포트폴리오에 있는 모든 종목 알려줘'
    """
    print(f"\n>>>> [호출] DB 조회 Agent (입력: {query})")
    app = await create_db_agent_graph() # 이전 단계에서 정의한 포트폴리오 DB 조회 에이전트를 재사용합니다.
    response = await app.ainvoke({"messages": [HumanMessage(content=query)], "turn_count": 0})
    print("<<<< [완료] DB 조회 Agent")
    return response["messages"][-1].content



# --- 3. 상위 Agent (Orchestrator) 워크플로우 구축 ---
async def create_orchestrator_graph():
    """
    'Agent + Reporter' 구조를 가진 상위 Orchestrator 그래프를 생성합니다.
    """
    # 상위 Agent가 사용할 도구 목록
    orchestrator_tools = [financial_info_agent_tool, portfolio_db_agent_tool]

    # 상위 Agent용 LLM 설정
    orchestrator_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0).bind_tools(orchestrator_tools)
    reporting_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

    # 상위 Agent의 상태 정의
    class OrchestratorState(TypedDict):
        messages: Annotated[List[BaseMessage], operator.add]
        turn_count: int

    # --- 노드 정의 ---
    def orchestrator_agent_node(state: OrchestratorState):
        """Orchestrator의 계획 및 도구 호출 담당 노드"""
        current_turn = state.get("turn_count", 0)
        print(f"\n\n++++++++++ 👑 ORCHESTRATOR AGENT (Turn {current_turn + 1}) ++++++++++")
        print("사용자의 복합 질문을 분석하고, 어떤 전문가 Agent를 호출할지 계획을 세웁니다...")
        system_prompt = """
        당신은 전문가 팀을 이끄는 마스터 Agent입니다. 당신의 임무는 사용자의 복잡한 요청을 해결하기 위해, 가장 적합한 전문가 Agent(도구)를 순서대로 호출하는 것입니다.
        **매우 중요한 규칙:**
        1. 당신은 도구를 반드시 **한 번에 하나씩만** 순서대로 호출해야 합니다.
        2. **포트폴리오의 모든 종목에 대한 정보 수집이 완료되었다고 판단되면, 더 이상 도구를 호출하지 말고 '모든 정보 수집 완료'라고만 답변하세요.**
        """
        response = orchestrator_llm.invoke([SystemMessage(content=system_prompt), *state["messages"]])
        return {"messages": [response], "turn_count": current_turn + 1}

    def orchestrator_report_node(state: OrchestratorState):
        """모든 하위 Agent의 결과물을 종합하여 최종 리포트를 생성하는 노드"""
        print("\n\n++++++++++ 💎 ORCHESTRATOR REPORTER ++++++++++")
        print("모든 전문가 Agent의 분석 결과를 종합하여 최종 리포트를 작성합니다...")
        report_history = [msg for msg in state["messages"] if not isinstance(msg, HumanMessage)]
        history_str = "\n".join([msg.pretty_repr() for msg in report_history])
        dedented_history = textwrap.dedent(history_str)
        initial_query = state["messages"][0].content

        report_prompt = f"""
        당신은 최고 투자 책임자(CIO)입니다. 아래는 각 팀(하위 Agent)이 수집하고 분석한 투자 정보입니다.

        [각 팀의 분석 보고 내용]
        {dedented_history}

        [초기 사용자 요청]
        {initial_query}

        위 정보를 모두 종합하여, 사용자의 초기 요청에 대한 최종 종합 투자 리포트를 상세하고 명확하게 작성해주세요.
        각 종목별 분석 내용을 먼저 요약하고, 마지막에 종합적인 결론과 투자 전략을 제시해주세요.
        """
        final_report = reporting_llm.invoke(report_prompt)
        return {"messages": [final_report]}

    # --- 라우터 정의 ---
    MAX_ORCHESTRATOR_TURNS = 10 # Orchestrator의 최대 실행 횟수 정의
    def orchestrator_router(state: OrchestratorState):
        """Orchestrator Agent의 다음 행동을 결정하는 라우터"""
        print("\n----- 🚦 ORCHESTRATOR ROUTER -----")

        # 최대 실행 횟수를 초과했는지 먼저 확인합니다.
        if state.get("turn_count", 0) >= MAX_ORCHESTRATOR_TURNS:
            print(f"⚠️ Safety Guard: 최대 반복 횟수({MAX_ORCHESTRATOR_TURNS}회)를 초과하여 강제로 리포트 생성을 시작합니다.\n")
            return "generate_report"

        last_message = state["messages"][-1]
        if last_message.tool_calls:
            print("분기: [call_tool] -> 하위 Agent(Tool)를 호출합니다.\n")
            return "call_tool"
        else:
            print("분기: [generate_report] -> 최종 리포트 생성을 시작합니다.\n")
            return "generate_report"

    # --- 그래프 구성 ---
    workflow = StateGraph(OrchestratorState)
    workflow.add_node("orchestrator_agent", orchestrator_agent_node)
    workflow.add_node("agent_as_tools", ToolNode(orchestrator_tools))
    workflow.add_node("reporter", orchestrator_report_node)

    workflow.set_entry_point("orchestrator_agent")
    workflow.add_edge("agent_as_tools", "orchestrator_agent")
    workflow.add_edge("reporter", END)

    # agent의 결과에 따라 tools로 가거나 reporter로 가도록 조건부 엣지 설정
    workflow.add_conditional_edges(
        "orchestrator_agent",
        orchestrator_router,
        {"call_tool": "agent_as_tools", "generate_report": "reporter"}
    )

    orchestrator_app = workflow.compile()
    print("\n🎉 'Agent + Reporter' 구조의 상위 Agent가 성공적으로 컴파일되었습니다.")
    return orchestrator_app


# --- 4. 상위 Agent 실행 ---
async def run_orchestrator():
    print("\n🚀 상위 Agent 실행을 시작합니다...")
    orchestrator_app = await create_orchestrator_graph()

    # --- 4-2. 시각화 ---
    display_mermaid_graph(orchestrator_app, title="Orchestrator (상위 Agent)")

    complex_query = "내 포트폴리오에 있는 모든 종목의 현재 상황에 대해 알려줘. 종목별 최신 주가와 부정적인 뉴스가 있는지 각각 확인해서 종합 리포트를 작성해줘."
    inputs = {"messages": [HumanMessage(content=complex_query)], "turn_count": 0}

    print("\n\n================ Orchestrator Agent 모델 실행 ================")
    print("\n++++++++++ 🙋‍♂️ USER INPUT ++++++++++")
    inputs["messages"][0].pretty_print()

    # 각 노드의 출력을 순서대로 처리
    async for update in orchestrator_app.astream(inputs, stream_mode="updates"):
        # update는 {'node_name': {'key': 'value'}} 형태의 딕셔너리
        node_name = list(update.keys())[0]
        node_output = list(update.values())[0]

        if node_name == "orchestrator_agent":
            # agent 노드의 AiMessage (계획 또는 최종 답변)를 출력
            print(f"\n--- 👑 ORCHESTRATOR AGENT ---")
            node_output['messages'][-1].pretty_print()

        elif node_name == "agent_as_tools":
            # tools 노드는 ToolMessage 리스트를 반환
            print("\n--- 🛠️ TOOLS ---")
            for tool_message in node_output['messages']:
                tool_message.pretty_print()

        elif node_name == "reporter":
            # reporter 노드의 최종 결과 AiMessage를 출력
            print("\n--- 💎 ORCHESTRATOR REPORTER ---")
            node_output['messages'][-1].pretty_print()

    print("\n✅ Orchestrator 실행이 완료되었습니다.")

# 비동기 함수 실행
await run_orchestrator()


🚀 상위 Agent 실행을 시작합니다...

🎉 'Agent + Reporter' 구조의 상위 Agent가 성공적으로 컴파일되었습니다.

📊 [ Orchestrator (상위 Agent) ]





================ Orchestrator Agent 모델 실행 ================

++++++++++ 🙋‍♂️ USER INPUT ++++++++++
================================ Human Message =================================

내 포트폴리오에 있는 모든 종목의 현재 상황에 대해 알려줘. 종목별 최신 주가와 부정적인 뉴스가 있는지 각각 확인해서 종합 리포트를 작성해줘.


++++++++++ 👑 ORCHESTRATOR AGENT (Turn 1) ++++++++++
사용자의 복합 질문을 분석하고, 어떤 전문가 Agent를 호출할지 계획을 세웁니다...

----- 🚦 ORCHESTRATOR ROUTER -----
분기: [call_tool] -> 하위 Agent(Tool)를 호출합니다.


--- 👑 ORCHESTRATOR AGENT ---
================================== Ai Message ==================================
Tool Calls:
  portfolio_db_agent_tool (30240c60-250a-4160-baf0-2bab98cb827c)
 Call ID: 30240c60-250a-4160-baf0-2bab98cb827c
  Args:
    query: 내 포트폴리오에 있는 모든 종목 알려줘

>>>> [호출] DB 조회 Agent (입력: 내 포트폴리오에 있는 모든 종목 알려줘)
📡 DB 서버에만 연결하는 MCP 클라이언트를 설정합니다...
✅ 포트폴리오 Agent용 도구 로딩 완료: ['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

++++++++++ 🤖 PORTFOLIO AGENT (Turn 1) ++++++++++
DB 조회를 위해 다음 행동을 결정합니다...

----- 🚦 ROUT

👇 아래 버튼을 눌러 코드를 실행합니다.

In [36]:
stop_mcp_server(port=8000, service_name="금융 정보")
stop_mcp_server(port=8001, service_name="포트폴리오 DB")


🛑 금융 정보 MCP 서버(Port: 8000)를 종료합니다...
✅ 금융 정보 서버가 종료되었습니다.

🛑 포트폴리오 DB MCP 서버(Port: 8001)를 종료합니다...
✅ 포트폴리오 DB 서버가 종료되었습니다.
